# Regular expressions

In this exercise, we will learn about parsing a text file using
regular expressions. We will analyse a Cisco log file using the `re`
module and extract log messages from the file.

## Regular expressions and the `re` module

Regular expressions are a powerful way to search, replace and parse
complex text.

They can also be cryptic, tricky and error-prone.

However, used well, they are more readable than a long chain of string
functions. They also tend to be implemented more efficiently by
optimised packages (such as Python's `re` module).

A good introduction to regular expressions with Python can be found here:

[http://www.diveintopython.net/regular_expressions/index.html](http://www.diveintopython.net/regular_expressions/index.html)

A more comprehensive tutorial for regular expressions is here:

[http://regex.learncodethehardway.org/book/](http://regex.learncodethehardway.org/book/)

For this exercise, you need to remember the following basics of
regular expressions. More details will be given in the following
exercises.

In [ ]:
^ matches the beginning of a string.
$ matches the end of a string.
. matches any character.
\b matches a word boundary.
\d matches any numeric digit.
\D matches any non-numeric character.
\s matches any whitespace character (tab, newline, etc.)
\S matches any non-whitespace character
\w matches any alphanumeric character (letters and numbers)
x? matches an optional x character (in other words, it matches an x zero or one times).
x* matches x zero or more times.
x+ matches x one or more times.
x{n,m} matches an x character at least n times, but not more than m times.
[a|b|c] matches exactly one of a, b or c.
(x) in general is a remembered group. You can get the value of what matched by using the groups() method of the object returned by re.search.

The `re` module is the standard module for regular expression
processing with Python. You can find the details of the module at
[https://docs.python.org/3/howto/regex.html](https://docs.python.org/3/howto/regex.html).

### Extract a Cisco IOS version number.

Throughout this session, we will analyse Cisco log messages in the
`show_tech.log` file.

Let's extract a version number of the Cisco IOS software from the log
file.

Here is a string example to parse.

In [2]:
message = 'Version 15.4(20141009:175959)'

In [3]:
import re
pattern = r"""
    Version     # start with Version
    \s+         # white space(s). \s for a white space, + for one or more times
    (\d+.\d+)   # the version number, xxx.xxx. () to create a group. \d for any numeric digit.
    \(          # left parenthesis
    (\d+):(\d+) # build numbers, xxx:xxx
    \)          # right parenthesis
    """
parser = re.compile(pattern, re.VERBOSE) # re.VERBOSE with verbose expression
searched = parser.search(message)
searched.groups()

('15.4', '20141009', '175959')

Please note that using `(x)` allows you to extracting values by
calling `groups()`. If you want to include a parenthesis in a search
pattern, prefix with backslash like `\(` and `\)`.

You can use a compact regular expression to define a pattern:

In [4]:
pattern = r'Version\s+(\d+.\d+)\((\d+):(\d+)\)'
parser = re.compile(pattern)
searched = parser.search(message)
searched.groups()

('15.4', '20141009', '175959')

Make sure to use a raw string prefixed with `r` when you use a compact
expression to prevent accidental conversion of escape characters
e.g. `'\t'` to a tab.

In this tutorial, we will use only verbose expressions for didactic
purposes. With additional comments, the verbose expression allows you
to put more details to each part of a complex pattern.

"Readability counts"

(Source: The Zen of Python at [https://www.python.org/dev/peps/pep-0020/](https://www.python.org/dev/peps/pep-0020/))

Unlike compact expressions, in a verbose regular expression,

- Whitespace is ignored. Spaces, tabs, and carriage returns are not
  matched. Instead use `\s` for white space matching.
- Comments are ignored.

Don't forget to use the `re.VERBOSE` flag as above when compiling a
verbose regular expression.

Now, using the parser, you can exact a version number of the Cisco IOS
Software directly from the following header in the log file.

In [ ]:
Cisco IOS XE Software, Version 2014-10-28_16.35_vipbansa
Cisco IOS Software, ASR1000 Software (X86_64_LINUX_IOSD-ADVIPSERVICESK9-M), Experimental Version 15.4(20141009:175959) [v154_3_s_xe313_throttle-vipbansa-V154_3_S_FC11-with-CSCun20543-removal 111]
Copyright (c) 1986-2014 by Cisco Systems, Inc.
Compiled Tue 28-Oct-14 15:24 by vipbansa

In [7]:
# Read the entire text and parse it with the pattern.
parser.search(open("/data/net/show_tech.log").read()).groups()

('15.4', '20141009', '175959')

#### Exercise

The `show_tech.log` file also has a version number for the Cisco IOS
XE Software. Create a regular expression pattern and extract the
version number from the log file.

In [1]:
!head -n50 /data/net/show_tech.log | tail -n6

***Hint:*** to match the username, use `\w` to match any alphanumeric
character (not a newline).

In [37]:
import re
pattern_xe = """
    Version     # start with Version
    \s+         # white space(s). \s for a white space, + for one or more times
    (\d+-\d+-\d+)   # the compile date in ISO 9660 format: YYYY-mm-dd
                    # () to create a group. \d for any numeric digit.
    _
    (\d+)\.(\d+) # other version numbers
    _
    (\w+)       # username who has compiled it (one or more letters)
    """

In [38]:
message = open("/data/net/show_tech.log").read()
parser_xe = re.compile(pattern_xe, re.VERBOSE | re.MULTILINE)
searched_xe = parser_xe.search(message)
searched_xe.groups()

('2014-10-28', '16', '35', 'vipbansa')

#### Exercise (b)

Try this example with `\D` instead of `\w` to match the username, to
see the difference. Read up on the `re.MULTILINE` regex compiler flag
in the Python "Regular Expressions HOWTO".


## Extract lines with specific log messages


The log file has several log messages that look like the following
string.

In [43]:
message = "*Jan  6 21:47:16.780: %LINEPROTO-5-UPDOWN: Line protocol on Interface TenGigabitEthernet0/1/0, changed state to up"

The first part of the log message, `'Jan 6 21:47:16.780'`, is the
recorded date and time of the log message.

The second part of the log message, `%LINEPROTO-5-UPDOWN`, is the
message code and severity level. In this message, the message code
family is `LINEPROTO`, the priority level is `5`, and a family type of
`UPDOWN`.

Each log message is categorised by the following severity types: `0`
for the most critical message and `7` for the least severe one.

![](extras/severity_levels.png)

So, `LINEPROTO-5-UPDOWN` is a simple notice.

And the final part is the message body, the human readable text of
each log message.

Using the following pattern, you can exact the message code from the
message.

In [44]:
pattern = """
    %           # start with %
    (           # defining a group
    [A-Z]+      # message class family
    \-          # - delimiter
    \d+         # severity code
    \-          # - delimiter
    [A-Z]+      # family type
    )           # closing a group
    """
parser = re.compile(pattern, re.VERBOSE)
searched = parser.search(message)
searched.groups()

('LINEPROTO-5-UPDOWN',)

Now with the parser, let's exact all error codes in the log
file. `findall()` will find all non-overlapping occurrences.

In [34]:
parser.findall(open("show_tech.log").read())

['CLNS-4-DUPSNPA',
 'LINK-3-UPDOWN',
 'LINK-3-UPDOWN',
 'LINEPROTO-5-UPDOWN',
 'LINK-3-UPDOWN',
 'LINK-3-UPDOWN',
 'LINEPROTO-5-UPDOWN',
 'LINEPROTO-5-UPDOWN',
 'LINK-3-UPDOWN',
 'LINEPROTO-5-UPDOWN',
 'SYS-5-CONFIG']

You can also extract all lines with log messages in this format.

In [46]:
for line in open("show_tech.log"):
    if parser.search(line): # note that None is treated as False in Python.
        print(line)

*Jan  2 22:03:35.210: %CLNS-4-DUPSNPA: ISIS: Duplicate SNPA 0024.976a.62bf detected

*Jan  3 03:27:45.444: %LINK-3-UPDOWN: Interface TenGigabitEthernet0/1/0, changed state to down

*Jan  3 03:27:45.443: %LINK-3-UPDOWN: SIP0/1: Interface TenGigabitEthernet0/1/0, changed state to down

*Jan  3 03:27:46.445: %LINEPROTO-5-UPDOWN: Line protocol on Interface TenGigabitEthernet0/1/0, changed state to down

*Jan  6 21:47:15.781: %LINK-3-UPDOWN: Interface TenGigabitEthernet0/1/0, changed state to up

*Jan  6 21:47:15.779: %LINK-3-UPDOWN: SIP0/1: Interface TenGigabitEthernet0/1/0, changed state to up

*Jan  6 21:47:16.780: %LINEPROTO-5-UPDOWN: Line protocol on Interface TenGigabitEthernet0/1/0, changed state to up

*Jan  6 22:20:16.653: %LINEPROTO-5-UPDOWN: Line protocol on Interface Port-channel20, changed state to down

*Jan  6 22:22:48.414: %LINK-3-UPDOWN: Interface Port-channel20, changed state to up

*Jan  6 22:22:49.413: %LINEPROTO-5-UPDOWN: Line protocol on Interface Port-channel20, chang

Now, let's extract each part of a log message line. Here is a string
to test our patterns.

In [47]:
message = "*Jan  6 21:47:16.780: %LINEPROTO-5-UPDOWN: Line protocol on Interface TenGigabitEthernet0/1/0, changed state to up"

In [123]:
pattern = """
    ^           # beginning of a string
    \*          # start with *
    (.*)        # date and time: match all chars greedily until
                # the rest of the expression matches
    :
    \s*
    %
    (           # message code
    [A-Z]+
    \-
    \d+
    \-
    [A-Z]+
    )
    :
    \s*
    (.*)        # message body
    """
parser = re.compile(pattern, re.VERBOSE)
searched = parser.search(message)
searched.groups()

('Jan  6 21:47:16.780',
 'LINEPROTO-5-UPDOWN',
 'Line protocol on Interface TenGigabitEthernet0/1/0, changed state to up')

The parser returns three components of the log message as a tuple. We
assign each component to a temporary variable for further parsing.

In [124]:
the_date, the_code, the_body = searched.groups()
the_date, the_code, the_body

('Jan  6 21:47:16.780',
 'LINEPROTO-5-UPDOWN',
 'Line protocol on Interface TenGigabitEthernet0/1/0, changed state to up')

### Exercise

The `show_tech.log` has the following messages about redundancy
history. Create a regular expression pattern and parse each line with
`RF_STATUS_REDUNDANCY_MODE_CHANGE`.

For instance, return `('00:16:39',
'RF_STATUS_REDUNDANCY_MODE_CHANGE(405)', 'LACP(226) op=7 rc=0')`

by parsing the following line:

In [ ]:
00:16:39 RF_STATUS_REDUNDANCY_MODE_CHANGE(405) LACP(226) op=7 rc=0

### Parsing each component of a log message further

Let's parse the first part, the date and time when each message was
logged.

In [52]:
the_date

'Jan  6 21:47:16.780'

In [54]:
pattern = """
    ^           # beginning of a string
    (\D{3})     # month
    \s+         # white space(s)
    (\d+)       # day
    \s+         # white space(s)
    (\d+)       # hour
    :           # delimiter :
    (\d+)       # minute
    :           # delimiter :
    (\d+)       # second
    \.          # floating point
    (\d+)       # a fraction of a second
    $           # end of a string
    """
parser = re.compile(pattern, re.VERBOSE)
searched = parser.search(the_date)
searched.groups()

('Jan', '6', '21', '47', '16', '780')

## Parsing dates and times

Often we may wish to convert this to a Python `datetime` object
instead of these strings. We can do this as follows:

In [58]:
import datetime

In [89]:
# See the docs for the `datetime.strptime()` function
# for the meaning of the codes below:
date_format = "%b  %d %H:%M:%S.%f"

In [90]:
the_date

'Jan  6 21:47:16.780'

In [107]:
my_date = datetime.datetime.strptime(the_date, date_format)

In [108]:
my_date

datetime.datetime(1900, 1, 6, 21, 47, 16, 780000)

Notice that this assumes the year 1900. To replace it with the current
year, we can use:

In [117]:
my_year = datetime.date.today().year

In [118]:
my_date.replace(my_year)

datetime.datetime(2015, 1, 6, 21, 47, 16, 780000)

If specifying the date/time format precisely like this seems too
laborious, you can use the 3rd-party `dateutil` package:

In [95]:
!pip install python-dateutil

Cleaning up...


Among its features, it offers "generic parsing of dates in almost any
string format":

In [119]:
from dateutil.parser import parse

In [120]:
parse(the_date)

datetime.datetime(2015, 1, 6, 21, 47, 16, 780000)

Now, we also break down the message code into three components.

In [121]:
the_code

'LINEPROTO-5-UPDOWN'

In [126]:
pattern = """
    ^           # beginning of a string
    ([A-Z]+)    # message code family
    -           # delimiter -
    (\d+)       # severity
    -           # delimiter -
    ([A-Z]+)    # family type
    $           # end of a string
"""
parser = re.compile(pattern, re.VERBOSE)
searched = parser.search(the_code)
searched.groups()

('LINEPROTO', '5', 'UPDOWN')

### Exercise


Extract the time and message parameters from each
`RF_STATUS_REDUNDANCY_MODE_CHANGE` message.

For instance, return the following tuple:

In [ ]:
('00', '16', '39', 'RF_STATUS_REDUNDANCY_MODE_CHANGE', '405', 'LACP', '226', 'op', '7', 'rc', '0')

from the following string:

In [ ]:
'00:16:39 RF_STATUS_REDUNDANCY_MODE_CHANGE(405) LACP(226) op=7 rc=0'

### Extended Exercise

In the data folder, one of the files is a private key. Using a regular
expression and `os.walk` or `glob`, find that file.

In [1]:
# See solutions/private_key.py"

## Other resources

- https://medium.com/@youknowjamest/parsing-file-names-using-regular-expressions-3e85d64deb69
- Understanding Regular Expressions (12 minute video): https://youtu.be/DRR9fOXkfRE
- Visualize Regular Expressions: https://regexr.com/
- Interactive Tutorial to learn Regular Expressions: https://regexone.com/
- Beginners Tutorial about Regular Expressions: https://www.analyticsvidhya.com/blog/2015/06/regular-expression-python/